# Draft Model

In [5]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import requests

In [4]:
BASE_URL = "https://www.fantasypros.com/nfl/adp/ppr-overall.php"

In [10]:
def make_adp_df():
    res = requests.get(BASE_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id':'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n',df.head(),'\n')
        df = df[['Player Team (Bye)','POS','AVG']]
        print('Output after filtering:\n\n',df.head(),'\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x:''.join(x.split()[:-2]))
        df['POS'] = df['POS'].apply(lambda x:x[:2])
        
        df = df[['PLAYER','POS','AVG']].sort_values(by='AVG')
        
        print('Final output:\n\n', df.head())
        
        return df
    
    else:
        print("Didn't work, try again")

In [11]:
df = make_adp_df()

Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      4.0  3.3
3     4           Dalvin Cook MIN (7)  RB4   4.0  NaN       5.0      5.0  4.7
4     5         Michael Thomas NO (6)  WR1   6.0  NaN       6.0      3.0  5.0 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.3
3           Dalvin Cook MIN (7)  RB4  4.7
4         Michael Thomas NO (6)  WR1  5.0 

Final output:

                PLAYER POS  AVG
0  ChristianMcCaffrey  RB  1.0
1       SaquonBarkley  RB  2.0
2      EzekielElliott  RB  3.3
3          DalvinCook  RB  4.7
4       MichaelThomas  WR  5.0


In [12]:
df.head()

,PLAYER,POS,AVG
0,ChristianMcCaffrey,RB,1.0
1,SaquonBarkley,RB,2.0
2,EzekielElliott,RB,3.3
3,DalvinCook,RB,4.7
4,MichaelThomas,WR,5.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1002 entries, 0 to 1001
Data columns (total 3 columns):
PLAYER    1002 non-null object
POS       1002 non-null object
AVG       1002 non-null float64
dtypes: float64(1), object(2)
memory usage: 31.3+ KB


In [ ]:
replacement_players = {
    'RB':,
    'WR':,
    'QB':,
    'TE':,
    
}